## Precarga de librerias y funciones

In [1]:
%%capture
%run "../../recurrentes.ipynb" 

In [2]:
%%capture
%run "../../funciones.ipynb"

#### Excel

In [3]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l
# d) dataset_10_meses_l

df_train = pd.read_parquet(dataset_clase_ternaria_l)

In [4]:
# df_train = df_train.drop(columns=dicc_psi['dataset_lags_deltas_y_clase_ternaria'])
# df_train = df_train.drop(columns=dicc_psi['lista_light_gbm_feature_importance'])

## Formateo pre modelo

In [5]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [6]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000
mes_train = [202011, 202012, 202101, 202102, 202103, 202104]
mes_test = 202106

In [7]:
data = df_train
data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
data['foto_mes'].unique()


df_train = data[data['foto_mes'].isin(mes_train)]
df_test = data[data['foto_mes']==mes_test]
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train['clase_peso']
w_test = df_test['clase_peso']

## Predicciones

In [8]:
# best_params = {'num_leaves': 1796,
#  'learning_rate': 0.049283676079631966,
#  'min_data_in_leaf': 9,
#  'feature_fraction': 0.44425115760554584,
#  'bagging_fraction': 0.40692971865213867}

In [9]:
best_params = {'num_leaves': 3545,
 'learning_rate': 0.07959540527561224,
 'min_data_in_leaf': 933,
 'feature_fraction': 0.6829932939284065,
 'bagging_fraction': 0.6251908881121132}
best_iter = 5

In [10]:
numeros_random = np.random.randint(0, 100000, 20)

In [11]:


train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)

In [ ]:

best_iter = 1789

df_voting = pd.DataFrame()

for x in semillas:
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': best_params['num_leaves'],
    'learning_rate': best_params['learning_rate'],
    'min_data_in_leaf': best_params['min_data_in_leaf'],
    'feature_fraction': best_params['feature_fraction'],
    'bagging_fraction': best_params['bagging_fraction'],
    'seed': x,
    'verbose': 0
}
        
            
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)

    y_pred = model.predict(X_test)
    df_voting[f'prediccion_seed_{x}'] = y_pred

df_voting['prediccion'] = df_voting.mean(axis=1)
df_voting.index = X_test.index



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
prediccion = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'probabilidad': df_voting ['prediccion']}, index=X_test.index) 
prediccion.shape 

(164876, 2)

In [ ]:
prediccion.isna().sum()

numero_de_cliente    0
probabilidad         0
dtype: int64

In [ ]:
bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']}, index= X_test.index)
bajas.shape

(164876, 2)

In [ ]:
bajas.isna().sum()

numero_de_cliente    0
clase_ternaria       0
dtype: int64

## Back Testing Bajas

In [ ]:
prediccion.shape

(164876, 2)

In [ ]:
merged_data = prediccion.merge(bajas, on='numero_de_cliente', how='inner')


In [ ]:
merged_data.columns

Index(['numero_de_cliente', 'prediccion', 'clase_ternaria'], dtype='object')

In [ ]:
merged_data_sorted = merged_data.sort_values('prediccion', ascending=False)


In [ ]:

# Ordenar por probabilidad de mayor a menor

# Seleccionar top clientes
top_clients = merged_data_sorted.iloc[:12000].copy()  # Trabajar con una copia

# Convertir 'clase_ternaria' a variable binaria para los top clientes
top_clients.loc[:, 'bajas_reales'] = (top_clients['clase_ternaria'] == 'BAJA+2').astype(int)

top_clients.value_counts('clase_ternaria')

clase_ternaria
CONTINUA    10816
BAJA+1        608
BAJA+2        576
Name: count, dtype: int64

In [ ]:
top_clients.value_counts('bajas_reales')

bajas_reales
0    11424
1      576
Name: count, dtype: int64

## Punto de Corte

In [ ]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)


pred_model_sorted = prediccion.sort_values('probabilidad', ascending=False)

model_name = 'LightGBM'

# Iteramos sobre cada corte
for corte in cortes:
    lista_ganancia_publico =[]
    lista_ganancia_privado = []
    for semilla in numeros_random: 
        ganancia_publico, ganancia_privado = calculoGanancia(bajas, pred_model_sorted, corte, semilla)        
    print(lista_ganancia_privado)
    print(lista_ganancia_publico)
    promedio_ganancia_publico = np.mean(lista_ganancia_publico)
    promedio_ganancia_privado = np.mean(lista_ganancia_privado)
        
    resultados.append({
    'Modelo': model_name,
    'Corte': corte,
    'Ganancia Público': ganancia_publico,
    'Ganancia Privado': ganancia_privado
    })

# Convertimos los resultados en un DataFrame
resultados = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

[68840000.0, 68580000.0, 69940000.0, 67960000.0, 70770000.0, 70860000.0, 64100000.00000001, 66580000.00000001, 68760000.0, 71800000.0]
[68040000.0, 68646666.66666667, 65473333.333333336, 70093333.33333334, 63536666.66666667, 63326666.66666667, 79100000.0, 73313333.33333334, 68226666.66666667, 61133333.333333336]
[71480000.0, 71210000.0, 69890000.0, 71060000.0, 72880000.0, 73200000.0, 67950000.0, 68750000.0, 70550000.0, 73230000.0]
[69346666.66666667, 69976666.66666667, 73056666.66666667, 70326666.66666667, 66080000.0, 65333333.333333336, 77583333.33333334, 75716666.66666667, 71516666.66666667, 65263333.333333336]
[76170000.0, 75180000.0, 75110000.0, 75190000.0, 79390000.0, 77550000.0, 71350000.0, 70750000.0, 75630000.0, 78190000.0]
[73336666.66666667, 75646666.66666667, 75810000.0, 75623333.33333334, 65823333.333333336, 70116666.66666667, 84583333.33333334, 85983333.33333334, 74596666.66666667, 68623333.33333334]
[77650000.0, 77410000.0, 79310000.0, 77470000.0, 80830000.0, 78570000.0, 

In [ ]:
# Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado']
)

# Aplanamos las columnas para facilitar el acceso
resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
columnas_ordenadas = []
for modelo in resultados['Modelo'].unique():
    columnas_ordenadas.append(f'Ganancia Público_{modelo}')
    columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# Reorganizamos el DataFrame usando el nuevo orden de columnas
resultados_pivot = resultados_pivot[columnas_ordenadas]

# Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
resultados_pivot = resultados_pivot.reset_index()
resultados_pivot

,Corte,Ganancia Público_LightGBM,Ganancia Privado_LightGBM
0,5000,6.113333e+07,71800000.0
1,6000,6.526333e+07,73230000.0
2,7000,6.862333e+07,78190000.0
3,8000,6.895000e+07,80450000.0
4,9000,7.324333e+07,83810000.0
5,10000,6.990667e+07,83640000.0
6,11000,7.417667e+07,81410000.0
7,12000,7.359333e+07,78860000.0
8,13000,7.315000e+07,78650000.0
9,14000,7.109667e+07,72730000.0


In [ ]:
maxima_ganancia_publico = resultados_pivot['Ganancia Público_LightGBM'].max()
maxima_ganancia_privado = resultados_pivot['Ganancia Privado_LightGBM'].max()


for x, corte, gan_publico, gan_privado in resultados_pivot.iterrows():
    if gan_publico == maxima_ganancia_publico:
        corte_maxima_ganancia_publico = corte
        ganancia_si_corte_maxima_ganancia_publico = gan_publico + gan_privado
        print (f'Ganancia si se hace corte por máxima ganancia público {corte_maxima_ganancia_publico}, el corte será {corte}')
    if gan_privado == maxima_ganancia_privado:
        corte_maxima_ganancia_privado = corte
        ganancia_si_corte_maxima_ganancia_privado= gan_publico + gan_privado
        print (f'Ganancia si se hace corte por máxima ganancia público {ganancia_si_corte_maxima_ganancia_privado}, el corte será {corte}')
               
corte_maxima_ganancia = corte_maxima_ganancia_publico if corte_maxima_ganancia_publico > corte_maxima_ganancia_privado else corte_maxima_ganancia_privado

ValueError: not enough values to unpack (expected 4, got 2)